In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
import arcgis
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
taz = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\MICROZONES\Inputs\TAZ.shp") 
buildings = pd.DataFrame.spatial.from_featureclass(r'E:\Projects\REMM-Manage-Base-Year-Data\Current_Inputs\remm_base_year.gdb\buildings')
roads = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\MICROZONES\Inputs\Roads.shp")

In [5]:
taz_wf = taz[taz['SUBAREAID']==1].copy()
taz_wf.spatial.to_featureclass(location=os.path.join(gdb, 'taz_wf'),sanitize_columns=False) 

'e:\\Tasks\\MICROZONES\\Outputs\\scratch.gdb\\taz_wf'

In [6]:
buildings.columns

Index(['OBJECTID', 'parcel_id', 'building_id', 'building_sqft',
       'building_type_id', 'non_residential_sqft', 'note', 'residential_units',
       'stories', 'unit_price_non_residential', 'year_built',
       'res_price_per_sqft', 'job_spaces', 'building_type_id2', 'basebldg',
       'building_type', 'SHAPE'],
      dtype='object')

In [7]:
# buildings = buildings[buildings['basebldg'] == 1].copy()
land_use = buildings[['parcel_id', 'building_type_id', 'building_type_id2', 'building_type', 'residential_units', 'basebldg', 'SHAPE']].copy()

lu = {0:'vacant',
 1:'residential', 
 2:'residential', 
 3:'industrial', 
 4:'commercial', 
 5:'commercial', 
 6:'govt', 
 7:'commercial', 
 8:'other', 
 9:'govt', 
 10:'govt', 
 11:'commercial', 
 12:'other', 
 13:'govt',
 14:'vacant',
 15:'vacant',
 16:'other',
 99:'vacant'
 }

land_use['land_use'] = land_use['building_type_id2'].map(lu)

land_use.loc[land_use['residential_units'] >= 8, 'land_use'] = 'multifamily'
land_use.spatial.to_featureclass(location=os.path.join(gdb, 'land_use'),sanitize_columns=False) 

# # calculate # acreage and filter small parcels
# gsa = arcgis.features.GeoSeriesAccessor(land_use['SHAPE'])  
# land_use['AREA'] = gsa.area * 0.000247105

# # land_use = land_use[land_use['AREA']> ]

'e:\\Tasks\\MICROZONES\\Outputs\\scratch.gdb\\land_use'

In [9]:
arcpy.management.Dissolve(os.path.join(gdb, 'land_use'), os.path.join(gdb, 'land_use_dissolved'), 'land_use', "", 
                          "SINGLE_PART", "DISSOLVE_LINES")

<Result '.\\Outputs\\scratch.gdb\\land_use_dissolved'>